In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/x_train.csv'
x_train = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/y_train.csv'
y_train = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/x_test.csv'
x_test = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/y_test.csv'
y_test = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Datasets/test.csv'
test = pd.read_csv(url)
url = 'https://media.githubusercontent.com/media/EricYangg/4AI3-Walmart-Forecast/main/Processed%20Dataset/Submission%20Test.csv'
submission_test = pd.read_csv(url)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.fit_transform(x_test)

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_train = x_train.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Temperature', 'MDMin', 'MDMax', 'MDSum'], axis=1)
x_train

,Store,Dept,Date,IsHoliday,Fuel_Price,CPI,Unemployment,MDAverage
0,1.392183,1.270486,-1.288570,-0.275106,-1.158147,-0.989963,-1.355282,-0.271794
1,-1.188910,-0.926810,-1.361242,-0.275106,-1.275919,0.466451,0.538139,-0.271794
2,-1.188910,0.352213,1.400316,-0.275106,0.455762,0.683805,-0.047383,-0.049038
3,0.140744,-1.221970,-0.125808,-0.275106,1.853757,-0.934874,0.135089,-0.271794
4,0.062529,-1.156379,-1.724605,-0.275106,-1.249747,-1.013147,-1.110017,-0.271794
...,...,...,...,...,...,...,...,...
330172,0.375389,-0.369287,1.424540,-0.275106,0.998821,-0.740830,0.149580,2.495277
330173,1.392183,-0.598856,-0.610292,-0.275106,-0.460240,-0.981996,-1.434711,-0.271794
330174,-0.641405,0.516190,1.109625,-0.275106,1.286708,0.503592,0.325612,0.842420
330175,-0.484975,-1.123583,-0.464947,-0.275106,-0.763393,0.525052,-0.722532,-0.271794


In [ ]:
param_grid = {'C': [1], 'gamma': [1e-4], 'kernel': ['rbf']}
grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=2)
grid.fit(x_train.to_numpy(), y_train.to_numpy().ravel())
print(grid.best_estimator_)
grid_predictions = grid.predict(x_test.to_numpy())

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .....................C=1, gamma=0.0001, kernel=rbf; total time=112.2min


In [ ]:
Id = test['Store'].astype('string') + '_' + test['Dept'].astype('string') + '_' + test['Date'].astype('string')
Id

In [ ]:
y_pred = grid.predict(submission_test)
y_pred

In [ ]:
weekly_sales = np.round(y_pred**3, 2)
weekly_sales

In [ ]:
sub = {'Id':Id, 'Weekly_Sales':weekly_sales}
SVR_submission = pd.DataFrame(data=sub)
SVR_submission

In [ ]:
Clf = SVR(C=1, gamma=0.1, kernel='rbf').fit(x_train,y_train)
train_score = Clf.score(x_train, y_train)
test_score = Clf.score(x_test, y_test)
test_score

In [ ]:
y_pred = Clf.predict(x_test)

In [ ]:
from sklearn.model_selection import learning_curve

train_size = [1, 25, 50, 100, 500, 2500, 7500, 15000]
train_sizes, train_scores, validation_scores = learning_curve(
estimator = Clf,
X = x_train,
y = y_train, train_sizes = train_size, cv = 5,
scoring = 'neg_mean_absolute_error')

In [ ]:
print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', validation_scores)

In [ ]:
train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = validation_scores.mean(axis=1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))

In [ ]:
train_scores_mean = -train_scores.mean(axis = 1)
validation_scores_mean = -validation_scores.mean(axis=1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
print('Mean Abs Error:', mean_absolute_error(y_test, y_pred))

In [ ]:
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training error')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
plt.ylabel('MAE', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for RANSAC regressor model', fontsize = 18, y = 1.03)
plt.legend()

In [ ]:
from google.colab import drive
drive.mount('drive')
SVR_submission.to_csv('SVR_submission.csv')
!cp SVR_submission.csv "drive/My Drive/"